In [2]:
import sys
sys.path.append(r"C:\Users\shrey")

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.converters import circuit_to_dag
from qiskit.transpiler import Target
from qiskit.circuit.library import QFT
from qiskit.circuit import Delay
from qiskit.circuit.library import XGate, YGate, ZGate
from qiskit.circuit import Gate
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info.operators import SparsePauliOp

from analytics.qubit_idling import analyze_qubit_idling, analyze_qubit_activity
from analytics.backend_characterization import extract_backend_metrics

In [4]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(name="Reserach_IITJ")
backend = service.backend("ibm_torino")

management.get:WARNING:2025-12-05 17:34:47,151: Loading saved account: Reserach_IITJ


In [5]:
from qiskit import QuantumCircuit
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import DynamicalDecoupling
from qiskit.circuit.library import XGate
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# --- 1️⃣ Normal preset transpiler ---
pm = generate_preset_pass_manager(
    optimization_level=3,
    seed_transpiler=42,
    backend=backend,
    scheduling_method='alap'
)

In [6]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes.scheduling import ALAPScheduleAnalysis
from qiskit.transpiler.passes import PadDynamicalDecoupling
from qiskit.transpiler import InstructionDurations
from qiskit.circuit.library import XGate

DD_SEQUENCE = [XGate(), XGate()]

DD_PM = PassManager(
    [
        ALAPScheduleAnalysis(backend.instruction_durations),
        PadDynamicalDecoupling(
            durations=backend.instruction_durations, dd_sequence=DD_SEQUENCE
        ),
    ]
)


def dynamical_decoupling_preprocess(
    input_circuit: QuantumCircuit, dd_pass_manager=DD_PM
) -> QuantumCircuit:
    """Apply dynamical decoupling to the input circuit.

    Args:
        input_circuit (QuantumCircuit): input circuit to run error mitigation on.
    """
    return dd_pass_manager.run(input_circuit)

In [7]:
from qiskit_aer import AerSimulator
simulator = AerSimulator()
backend_sim = AerSimulator.from_backend(backend)

In [8]:
from qiskit_ibm_runtime import EstimatorV2, EstimatorOptions

estimator_op = EstimatorOptions()
estimator_op.resilience_level = 0
estimator_op.dynamical_decoupling.enable = False
estimator_op.twirling.enable_gates = False
estimator_op.twirling.enable_measure = False

estimator_no_dd = EstimatorV2(mode = backend, options=estimator_op)

In [9]:
estimator_op = EstimatorOptions()
estimator_op.resilience_level = 0
estimator_op.dynamical_decoupling.enable = True
estimator_op.twirling.enable_gates = False
estimator_op.twirling.enable_measure = False

estimator_dd = EstimatorV2(mode = backend, options=estimator_op)

In [10]:
idle_estimator = EstimatorV2(mode = simulator)

In [11]:
all_circuits_df = pd.DataFrame(columns=[
    "circuit_id", "qubit",
    "sparsity_by_layer", "sparsity_by_time",
    "mean_dt", "sd_dt",
    "max_delay_dt", "decoher_err_prob",
    "marginal_exp_error_no_dd", "marginal_exp_error_dd", "marginal_improvment",
    "expectation_error_no_dd", "expectation_error_dd", "overall_improvment"
])

# --- Function to store metrics for one circuit ---
def store_circuit_results(qubit_act, qubit_idle, marginal_exp_error_no_dd, marginal_exp_error_dd, expectation_error_no_dd, expectation_error_dd, circuit_id):
    rows = []
    for qubit in qubit_act.get("sparsity_by_layer", {}).keys():
        row = {
            "circuit_id": circuit_id,
            "qubit": qubit,
            # From qubit activity
            "sparsity_by_layer": qubit_act["sparsity_by_layer"].get(qubit, None),
            "sparsity_by_time": qubit_act.get("sparsity_by_time", {}).get(qubit, None),
            "mean_dt": qubit_act.get("distribution_mean_dt", {}).get(qubit, None),
            "sd_dt": qubit_act.get("distribution_sd_dt", {}).get(qubit, None),
            # From qubit idling (decoherence)
            "max_delay_dt": qubit_idle["max_ratio_qubit"].get("max_delay_dt", None),
            "decoher_err_prob": qubit_idle["max_ratio_qubit"].get("decoher_err_prob", None),
            "marginal_exp_error_no_dd": marginal_exp_error_no_dd.get(qubit, None),
            "marginal_exp_error_dd": marginal_exp_error_dd.get(qubit, None),
            "marginal_improvment": marginal_exp_error_no_dd.get(qubit, 0) - marginal_exp_error_dd.get(qubit, 0),
            "expectation_error_no_dd": expectation_error_no_dd,
            "expectation_error_dd": expectation_error_dd,
            "overall_improvment": expectation_error_no_dd - expectation_error_dd
        }
        rows.append(row)
    return pd.DataFrame(rows)

In [12]:
def ghz_circuit(num_qubits, measure=True):
    """
    Generate a GHZ state circuit with num_qubits qubits.
    If measure=True, adds measurements to all qubits.
    """
    qc = QuantumCircuit(num_qubits, num_qubits if measure else 0)
    
    # Step 1: Apply H on the first qubit
    qc.h(0)
    
    # Step 2: Apply a chain of CX gates
    for i in range(num_qubits - 1):
        qc.cx(i, i + 1)
    
    qc.barrier()

    # Step 3: Optional measurement
    if measure:
        qc.measure(range(num_qubits), range(num_qubits))
    
    return qc

def echo_circuit(n):
    # Create GHZ state
    ghz = QuantumCircuit(n)
    ghz.h(0)
    for i in range(n - 1):
        ghz.cx(i, i + 1)

    # Loschmidt echo circuit: GHZ → barrier → GHZ† → measurement
    echo = QuantumCircuit(n)
    echo.compose(ghz, inplace=True)
    echo.barrier()
    echo.compose(ghz.inverse(), inplace=True)
    echo.x(range(n)) 
    echo.measure_all()

    return echo

In [13]:
def compute_expectation(num_qubits, quantum_circuit, name):
    transpiled_circ = pm.run(quantum_circuit)
    transpiled_circ_dd = dynamical_decoupling_preprocess(transpiled_circ)

    backend_prop = extract_backend_metrics(transpiled_circ, backend)
    qubit_act = analyze_qubit_activity(transpiled_circ, backend.target)
    qubit_idle = analyze_qubit_idling(transpiled_circ, backend)

    observables = []
    observables.append(SparsePauliOp('Z'*num_qubits, coeffs=[1.0]))
    for i in range(num_qubits):
        obs = SparsePauliOp('I'*i + 'Z' + 'I'*(num_qubits - i -1), coeffs=[1.0])
        observables.append(obs)
    transpiled_observables = [ob.apply_layout(transpiled_circ.layout) for ob in observables]

    # --- Run Sampler and Estimator (example) ---
    idle_exp = idle_estimator.run([(transpiled_circ, transpiled_observables)]).result()[0].data.evs
    print("Idle expectations:", idle_exp)
    exp_no_dd = estimator_no_dd.run([(transpiled_circ, transpiled_observables)]).result()[0].data.evs
    exp_dd = estimator_dd.run([(transpiled_circ_dd, transpiled_observables)]).result()[0].data.evs

    layout = transpiled_circ.layout.final_index_layout()
    marginal_exp_error_no_dd = {}
    marginal_exp_error_dd = {}

    for i in range(num_qubits):
        marginal_exp_error_no_dd[layout[i]] = np.abs(exp_no_dd[i+1]- idle_exp[i+1])
        marginal_exp_error_dd[layout[i]] = np.abs(exp_dd[i+1]- idle_exp[i+1])
    
    df_first_circuit = store_circuit_results(qubit_act, qubit_idle, marginal_exp_error_no_dd, marginal_exp_error_dd, np.abs(exp_no_dd[0]-idle_exp[0]), np.abs(exp_dd[0]-idle_exp[0]), name)
    return df_first_circuit

In [14]:
num_qubits_list = [2,7, 12, 15, 30]
for num_qubits in num_qubits_list:
    rand_circ = ghz_circuit(num_qubits)
    df_first_circuit = compute_expectation(num_qubits, rand_circ, f"GHZ_{num_qubits}")
    all_circuits_df = pd.concat([all_circuits_df, df_first_circuit], ignore_index=True)

Idle expectations: [1.         0.00146484 0.00146484]


C:\Users\shrey\AppData\Local\Temp\ipykernel_21468\2812277530.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_circuits_df = pd.concat([all_circuits_df, df_first_circuit], ignore_index=True)


Idle expectations: [0.01757812 0.01757812 0.01757812 0.01757812 0.01757812 0.01757812
 0.01757812 0.01757812]
Idle expectations: [1.         0.00830078 0.00830078 0.00830078 0.00830078 0.00830078
 0.00830078 0.00830078 0.00830078 0.00830078 0.00830078 0.00830078
 0.00830078]
Idle expectations: [-0.00732422 -0.00732422 -0.00732422 -0.00732422 -0.00732422 -0.00732422
 -0.00732422 -0.00732422 -0.00732422 -0.00732422 -0.00732422 -0.00732422
 -0.00732422 -0.00732422 -0.00732422 -0.00732422]
Idle expectations: [ 1.         -0.02099609 -0.02099609 -0.02099609 -0.02099609 -0.02099609
 -0.02099609 -0.02099609 -0.02099609 -0.02099609 -0.02099609 -0.02099609
 -0.02099609 -0.02099609 -0.02099609 -0.02099609 -0.02099609 -0.02099609
 -0.02099609 -0.02099609 -0.02099609 -0.02099609 -0.02099609 -0.02099609
 -0.02099609 -0.02099609 -0.02099609 -0.02099609 -0.02099609 -0.02099609
 -0.02099609]


In [15]:
num_qubits_list = [2,7, 12, 15,30]
for num_qubits in num_qubits_list:
    rand_circ = echo_circuit(num_qubits)
    df_first_circuit = compute_expectation(num_qubits, rand_circ, f"Echo_{num_qubits}")
    all_circuits_df = pd.concat([all_circuits_df, df_first_circuit], ignore_index=True)

Idle expectations: [ 1. -1. -1.]
Idle expectations: [-1. -1. -1. -1. -1. -1. -1. -1.]
Idle expectations: [ 1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
Idle expectations: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
Idle expectations: [ 1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]


In [16]:
all_circuits_df.to_csv("E:\\Desktop\\QAMP results\\Torino_estimator_051225.csv")